In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
import os
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [2]:
# Define the directory path containing the images
data_directory = "/Users/ouzibigouziouzi/Downloads/data"
batch_size = 32
image_size = (200,200)
data = tf.keras.utils.image_dataset_from_directory(
    data_directory,
    batch_size=batch_size,
    image_size = image_size,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="training")

Found 4188 files belonging to 4 classes.
Using 3351 files for training.


In [3]:
# Print the class names
class_names = data.class_names
print("Class names:", class_names)

Class names: ['Blight', 'Common_Rust', 'Gray_Leaf_Spot', 'Healthy']


In [4]:
# Convert labels to NumPy arrays
X_train = np.concatenate([x for x, _ in data], axis=0)
y_train = np.concatenate([y for _, y in data], axis=0)

In [5]:
# Convert labels to one-hot encoded format
y_train = to_categorical(y_train, num_classes=4)

In [7]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dropout

In [8]:
model = tf.keras.Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dense(4, activation='softmax')
])


# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 99, 99, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 48, 48, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 23, 23, 64)        0

In [9]:
# Train the model
model.fit(X_train, y_train, epochs=11)

Epoch 1/11
105/105 [==============================] - 24s 220ms/step - loss: 24.4854 - accuracy: 0.2844
Epoch 2/11
105/105 [==============================] - 20s 195ms/step - loss: 2.9612 - accuracy: 0.3590
Epoch 3/11
105/105 [==============================] - 19s 182ms/step - loss: 2.6775 - accuracy: 0.4196
Epoch 4/11
105/105 [==============================] - 19s 179ms/step - loss: 2.4429 - accuracy: 0.4837
Epoch 5/11
105/105 [==============================] - 19s 183ms/step - loss: 2.2285 - accuracy: 0.5333
Epoch 6/11
105/105 [==============================] - 19s 178ms/step - loss: 2.0745 - accuracy: 0.5688
Epoch 7/11
105/105 [==============================] - 19s 179ms/step - loss: 1.9564 - accuracy: 0.6043
Epoch 8/11
105/105 [==============================] - 19s 183ms/step - loss: 1.8486 - accuracy: 0.6335
Epoch 9/11
105/105 [==============================] - 19s 185ms/step - loss: 1.7526 - accuracy: 0.6619
Epoch 10/11
105/105 [==============================] - 19s 179ms/step - 

In [10]:
val_data = tf.keras.utils.image_dataset_from_directory(
    data_directory,
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="validation"
)

Found 4188 files belonging to 4 classes.
Using 837 files for validation.


In [11]:
# Convert labels to NumPy arrays
X_val = np.concatenate([x for x, _ in val_data], axis=0)
y_val = np.concatenate([y for _, y in val_data], axis=0)

In [12]:
# Convert labels to one-hot encoded format
y_val = to_categorical(y_val, num_classes=4)

In [13]:
# Evaluate the model on the validation data
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

27/27 [==============================] - 2s 50ms/step - loss: 3.1521 - accuracy: 0.2832
Validation Loss: 3.152056932449341
Validation Accuracy: 0.28315412998199463


In [14]:
# Define the function to create the model
def create_model(optimizer='adam', kernel_regularizer=None):
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', input_shape=(200, 200, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu', kernel_regularizer=kernel_regularizer),
        Dense(4, activation='softmax')
    ])
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [15]:
# Convert labels to one-hot encoded format
y_train_categorical = to_categorical(y_train, num_classes=4)

In [17]:
from scikeras.wrappers import KerasClassifier

In [18]:
# Create the KerasClassifier wrapper
model = KerasClassifier(build_fn=create_model)


In [19]:
from sklearn.model_selection import GridSearchCV


In [20]:
# Define the function to create the model
def create_model(optimizer='adam'):
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', input_shape=(200, 200, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(4, activation='softmax')
    ])
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model
# Create the KerasClassifier for GridSearchCV
model = KerasClassifier(build_fn=create_model, epochs=10, verbose=0)

# Define the hyperparameters to tune
param_grid = {
    'optimizer': ['adam', 'sgd'],
    'batch_size': [16, 32, 64],
    'epochs': [10, 20, 30],
}

# Perform GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train)

# Print the best score and best parameters

/Users/ouzibigouziouzi/.pyenv/versions/3.11.3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/ouzibigouziouzi/.pyenv/versions/3.11.3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/ouzibigouziouzi/.pyenv/versions/3.11.3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/ouzibigouziouzi/.pyenv/versions/3.11.3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at w

In [21]:
# Print the best score and best parameters
print("Best Score:", grid_result.best_score_)
print("Best Parameters:", grid_result.best_params_)

Best Score: 0.2924500149209191
Best Parameters: {'batch_size': 16, 'epochs': 10, 'optimizer': 'sgd'}
